In [1]:
from openai import AzureOpenAI

api_key = "a1ab70e0f13c4db1b2236ae9ea986511"
api_version = "2024-05-01-preview"

assistantClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint="https://james-ai.openai.azure.com/openai/assistants?api-version=2024-05-01-preview",
    api_version=api_version,
)

# Create an assistant
assistant = assistantClient.beta.assistants.create(
    name="Math Assistant",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o"
)

assistant

Assistant(id='asst_NscqprMxyvhxiSVVikyJgBj2', created_at=1716653384, description=None, instructions='You are an AI assistant that can write code to help answer math questions.', metadata={}, model='gpt-4o', name='Math Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=[]), file_search=None), top_p=1.0)

In [2]:
threadsClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint="https://james-ai.openai.azure.com/openai/threads?api-version=2024-05-01-preview",
    api_version=api_version,
)

# Create a thread
thread = threadsClient.beta.threads.create()

thread

Thread(id='thread_DqvZqrBFOQYOSVXFIru2pouT', created_at=1716653385, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [3]:
messageClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=f"https://james-ai.openai.azure.com/openai/threads/{thread.id}/messages?api-version=2024-05-01-preview",
    api_version=api_version,
)

# Add a user question to the thread
message = messageClient.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me create a simple C program to solve this?"
)

message

Message(id='msg_oCmMN60rA7lQNdOsZGucW94x', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me create a simple C program to solve this?'), type='text')], created_at=1716653387, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_DqvZqrBFOQYOSVXFIru2pouT')

In [4]:
import time

runClient = AzureOpenAI(
    api_key=api_key,
    azure_endpoint=f"https://james-ai.openai.azure.com/openai/threads/{thread.id}/runs?api-version=2024-05-01-preview",
    api_version=api_version,
)

# Run the thread and poll for the result
run = runClient.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Please address the user as Jane Doe. The user has a premium account.",
)

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  print(run.status)
  run = runClient.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

queued


In [6]:
messages = messageClient.beta.threads.messages.list(
  thread_id=thread.id
)

messages.data[0]

Message(id='msg_yM1jJswBM5cgp8llhlg5uz1p', assistant_id='asst_NscqprMxyvhxiSVVikyJgBj2', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Of course, Jane Doe! Below is a simple C program to solve the equation \\(3x + 11 = 14\\). The program will calculate and display the value of \\(x\\).\n\n```c\n#include <stdio.h>\n\nint main() {\n    // Define variables\n    float x;\n    int a = 3;\n    int b = 11;\n    int c = 14;\n\n    // Calculate x\n    x = (float)(c - b) / a;\n\n    // Display the result\n    printf("The solution to the equation 3x + 11 = 14 is x = %.2f\\n", x);\n\n    return 0;\n}\n```\n\nYou can copy and paste this code into your C IDE or text editor, and then compile and run the program. It will display the value of \\(x\\) that satisfies the equation \\(3x + 11 = 14\\).'), type='text')], created_at=1716653390, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_SBy5im